In [1]:
import numpy as np # linear algebra
import pandas as pd
import joblib

In [3]:
#load data
df_titanic = pd.read_csv('titanic.csv')

In [4]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df_clear=pd.read_csv('df_titanic_clear.csv')

In [35]:
df_clear.head()

,Survived,Pclass,Fare_dbscan,Sex_enc,Embarked_enc,Mr,Mrs,Miss,Master,Other,young_adult,adult,elderly,child
0,0,3,7.2500,0,2,1,0,0,0,0,1,0,0,0
1,1,1,71.2833,1,0,0,1,0,0,0,0,1,0,0
2,1,3,7.9250,1,2,0,0,1,0,0,1,0,0,0
3,1,1,53.1000,1,2,0,1,0,0,0,0,1,0,0
4,0,3,8.0500,0,2,1,0,0,0,0,0,1,0,0


In [40]:
# df_clear['Fare_dbscan'].unique()

In [ ]:
# Survived : 0 : dead, 1 : alive
# Pclass : 1 , 2 , 3
# sex_enc : 0 :male, 1:female
# Embarked  0:C, 1:Q , 2:S
# Mr, Mrs, Miss, Other 
                'Mlle': 'Miss',
                 'Mme': 'Miss',
                 'Ms': 'Miss',
                 'Dr': 'Mr',
                 'Major': 'Mr',
                 'Lady': 'Mrs',
                 'Countess': 'Mrs',
                 'Jonkheer': 'Other',
                 'Col': 'Other',
                 'Rev': 'Other',
                 'Capt': 'Mr',
                 'Sir': 'Mr',
                 'Don': 'Mr'
                    
# child  : 0-15
# young_adult: 16-30
# adult : 31 -50
# ederly :51-100




In [10]:
from sklearn.model_selection import train_test_split

# Memisahkan fitur dan label dari data Anda
X = df_clear.drop('Survived', axis=1)
y = df_clear['Survived']
# Memisahkan data menjadi set pelatihan dan set pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X, y)

feature_importances = model.feature_importances_
fitur=[]
nilai=[]
for feature, importance in zip(X.columns, feature_importances):
    fitur.append(feature)
    nilai.append(importance)

In [12]:
feature_importan= pd.DataFrame({'Fiture':fitur, 'Nilai':nilai})
feature_importan.sort_values(by='Nilai', ascending=False)

,Fature,Nilai
1,Fare_dbscan,0.410668
4,Mr,0.137102
2,Sex_enc,0.132619
0,Pclass,0.123231
6,Miss,0.045815
3,Embarked_enc,0.044311
5,Mrs,0.040812
10,adult,0.014441
9,young_adult,0.013016
12,child,0.012173


In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [5,10,50],
               'max_depth': [5,10,20],
               'min_samples_split': [2,5,10],
               'min_samples_leaf': [2,5,10],
               'bootstrap': [False],
                'criterion': ['gini']}
gs = GridSearchCV(model, param_grid=param_grid, cv = 3, verbose=1, n_jobs=-1)
gs.fit(X, y)
print(gs.best_estimator_)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
RandomForestClassifier(bootstrap=False, max_depth=20, min_samples_leaf=2,
                       min_samples_split=10, n_estimators=5)


In [14]:
predictions = gs.best_estimator_.predict(X_test)

In [20]:
actual=y_test

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

conf_matrix = confusion_matrix(actual, predictions)



print(conf_matrix)
print('Accuracy score:', round(accuracy_score(actual, predictions),2))
print('Recall score:', round(recall_score(actual, predictions),2))
print('Precision score:', round(precision_score(actual, predictions),2))
print('F1 score:', round(f1_score(actual, predictions),2))

[[96  9]
 [ 9 65]]
Accuracy score: 0.9
Recall score: 0.88
Precision score: 0.88
F1 score: 0.88


In [22]:
model_rf= RandomForestClassifier(bootstrap=False, max_depth=20, min_samples_leaf=2,
                       min_samples_split=10, n_estimators=5)

In [26]:
joblib.dump(model_rf, 'model.joblib')

['model.joblib']

In [30]:
model_loaded = joblib.load('model.joblib')

In [31]:
X[:1]

,Pclass,Fare_dbscan,Sex_enc,Embarked_enc,Mr,Mrs,Miss,Master,Other,young_adult,adult,elderly,child
0,3,7.25,0,2,1,0,0,0,0,1,0,0,0


In [32]:
y_predict = model_loaded.predict(X[1:2])
y_predict

array([1])

# Probabilitas

In [33]:
pred_prob = model_loaded.predict_proba(X[:1])

In [34]:
pred_prob

array([[0.98036364, 0.01963636]])